In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from pprint import pprint
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError, SparseCategoricalCrossentropy # type: ignore
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('archive/Student_performance_data _.csv')
data = data.iloc[:,1:].to_numpy()
normalizer = Normalization(axis=-1)

Y = data[:,-1]
X = data[:,:-2]
x_train, x_, y_train, y_ = train_test_split(X,Y, test_size=0.4, random_state=1)
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)
del x_, y_

normalizer.adapt(x_train)
normalized_x_train = normalizer(x_train)
normalizer.adapt(x_cv)
normalized_x_cv = normalizer(x_cv)
normalizer.adapt(x_test)
normalized_x_test = normalizer(x_test)

In [3]:
def eval_cat_err(y, yhat):
    """ 
    Calculate the categorization error
    Args:
      y    : (ndarray  Shape (m,) or (m,1))  target value of each example
      yhat : (ndarray  Shape (m,) or (m,1))  predicted value of each example
    Returns:|
      cerr: (scalar)             
    """
    m = len(y)
    incorrect = 0
    for i in range(m):
    ### START CODE HERE ### 
        if y[i] != yhat[i]:
           incorrect += 1
    
    cerr = incorrect / m
    
    ### END CODE HERE ### 
    
    return(cerr)

In [4]:
tf.random.set_seed(1234)
model = Sequential(
    [
        ### START CODE HERE ### 
        Dense(units=120, activation='relu'),
        Dense(units=40, activation='relu'),
        Dense(units=5, activation='linear')
        ### END CODE HERE ### 

    ], name="Complex"
)
model.compile(
    ### START CODE HERE ### 
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(0.01),
    ### END CODE HERE ### 
)

In [5]:
model.fit(normalized_x_train,y_train,epochs=500, batch_size=64)

Epoch 1/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.2515   
Epoch 2/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - loss: 0.8432
Epoch 3/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step - loss: 0.7599
Epoch 4/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - loss: 0.6908
Epoch 5/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - loss: 0.6465
Epoch 6/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - loss: 0.6087
Epoch 7/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - loss: 0.5728
Epoch 8/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - loss: 0.5448
Epoch 9/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - loss: 0.5273
Epoch 10/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - loss: 0.4911
Epoch 11/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 599us/step - loss: 0.4651
Epoch 12/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4385 
Epoch 13/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - loss: 0.4125
Epoch 14/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - loss: 0.3978
Epoch 15/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [6]:
model_predict = lambda Xl: np.argmax(tf.nn.softmax(model.predict(Xl)).numpy(),axis=1)
training_cerr_complex = eval_cat_err(y_train, model_predict(normalized_x_train))
cv_cerr_complex = eval_cat_err(y_cv, model_predict(normalized_x_cv))
print(f"categorization error, training, complex model: {training_cerr_complex:0.3f}")
print(f"categorization error, cv,       complex model: {cv_cerr_complex:0.3f}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
categorization error, training, complex model: 0.026
categorization error, cv,       complex model: 0.335


In [7]:
tf.random.set_seed(1234)
model_s = Sequential(
    [
        ### START CODE HERE ### 
        Dense(units=5, activation='relu'),
        Dense(units=5, activation='linear')
        ### END CODE HERE ### 
    ], name = "Simple"
)
model_s.compile(
    ### START CODE HERE ### 
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(0.01),
    ### START CODE HERE ### 
)

In [8]:
# BEGIN UNIT TEST
model_s.fit(
    normalized_x_train,y_train,
    epochs=500, batch_size=64
)
# END UNIT TEST

Epoch 1/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 648us/step - loss: 1.8106 
Epoch 2/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - loss: 1.2914
Epoch 3/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - loss: 1.1119
Epoch 4/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - loss: 0.9729
Epoch 5/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - loss: 0.8918
Epoch 6/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - loss: 0.8480
Epoch 7/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - loss: 0.8218
Epoch 8/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - loss: 0.8031
Epoch 9/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - loss: 0.7898
Epoch 10/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - loss: 0.7805
Epoch 11/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - loss: 0.7729
Epoch 12/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - loss: 0.7662
Epoch 13/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 0.7612
Epoch 14/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - loss: 0.7570
Epoch 15/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s

In [9]:
model_predict_s = lambda Xl: np.argmax(tf.nn.softmax(model_s.predict(Xl)).numpy(),axis=1)
training_cerr_simple = eval_cat_err(y_train, model_predict_s(normalized_x_train))
cv_cerr_simple = eval_cat_err(y_cv, model_predict_s(normalized_x_cv))
print(f"categorization error, training, simple model, {training_cerr_simple:0.3f}, complex model: {training_cerr_complex:0.3f}" )
print(f"categorization error, cv,       simple model, {cv_cerr_simple:0.3f}, complex model: {cv_cerr_complex:0.3f}" )

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step
categorization error, training, simple model, 0.226, complex model: 0.026
categorization error, cv,       simple model, 0.268, complex model: 0.335


In [10]:
tf.random.set_seed(1234)
model_r = Sequential(
    [
        ### START CODE HERE ### 
        Dense(units=120, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)),
        Dense(units=40, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)),
        Dense(units=5, activation='linear')        
        ### START CODE HERE ### 
    ], name= None
)
model_r.compile(
    ### START CODE HERE ### 
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(0.01),
    ### START CODE HERE ### 
)

In [11]:
# BEGIN UNIT TEST
model_r.fit(
    normalized_x_train,y_train,
    epochs=500, batch_size=64
)
# END UNIT TEST

Epoch 1/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 693us/step - loss: 5.5740 
Epoch 2/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - loss: 1.3005
Epoch 3/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step - loss: 1.1208
Epoch 4/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 1.0954
Epoch 5/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - loss: 1.0800
Epoch 6/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - loss: 1.0351
Epoch 7/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 577us/step - loss: 1.0131
Epoch 8/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - loss: 1.0088
Epoch 9/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - loss: 1.0031
Epoch 10/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - loss: 0.9891
Epoch 11/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 0.9751
Epoch 12/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - loss: 0.9642
Epoch 13/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - loss: 0.9551
Epoch 14/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step - loss: 0.9472
Epoch 15/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s

In [12]:
model_predict_r = lambda Xl: np.argmax(tf.nn.softmax(model_r.predict(Xl)).numpy(),axis=1)
training_cerr_reg = eval_cat_err(y_train, model_predict_r(normalized_x_train))
cv_cerr_reg = eval_cat_err(y_cv, model_predict_r(normalized_x_cv))
test_cerr_reg = eval_cat_err(y_test, model_predict_r(normalized_x_test))
print(f"categorization error, training, regularized: {training_cerr_reg:0.3f}, simple model, {training_cerr_simple:0.3f}, complex model: {training_cerr_complex:0.3f}" )
print(f"categorization error, cv,       regularized: {cv_cerr_reg:0.3f}, simple model, {cv_cerr_simple:0.3f}, complex model: {cv_cerr_complex:0.3f}" )

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step
categorization error, training, regularized: 0.233, simple model, 0.226, complex model: 0.026
categorization error, cv,       regularized: 0.266, simple model, 0.268, complex model: 0.335


In [13]:
tf.random.set_seed(1234)
lambdas = [0.0, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
models=[None] * len(lambdas)

for i in range(len(lambdas)):
    lambda_ = lambdas[i]
    models[i] =  Sequential(
        [
            Dense(120, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
            Dense(40, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
            Dense(5, activation = 'linear')
        ]
    )
    models[i].compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(0.01),
    )

    models[i].fit(
        normalized_x_train,y_train,
        epochs=1000
    )

for i in range(len(models)):
    model_predict = lambda Xl: np.argmax(tf.nn.softmax(models[i].predict(Xl)).numpy(),axis=1)
    print(f"lambda: {lambdas[i]}, train err {eval_cat_err(y_train, model_predict(normalized_x_train))},cv err {eval_cat_err(y_cv, model_predict(normalized_x_cv))},test err {eval_cat_err(y_test, model_predict(normalized_x_test))}")

Epoch 1/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 630us/step - loss: 1.2265 
Epoch 2/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - loss: 0.8388
Epoch 3/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - loss: 0.7425
Epoch 4/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - loss: 0.6873
Epoch 5/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - loss: 0.6561
Epoch 6/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - loss: 0.6222
Epoch 7/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - loss: 0.5855
Epoch 8/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - loss: 0.5591
Epoch 9/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - loss: 0.5149
Epoch 10/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 0.4953
Epoch 11/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - loss: 0.4786
Epoch 12/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - loss: 0.4539
Epoch 13/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step - loss: 0.4261
Epoch 14/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - loss: 0.4241
Epoch 15/1000
45/45 ━━━━━━━━

In [14]:
for model in models:
    model_predict = lambda Xl: np.argmax(tf.nn.softmax(model.predict(Xl)).numpy(),axis=1)
    print(sum(y_train != model_predict(normalized_x_train)) / len(y_train),
          sum(y_cv != model_predict(normalized_x_cv)) / len(y_cv),
          sum(y_test != model_predict(normalized_x_test)) / len(y_test))

 1/45 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step
0.0020905923344947735 0.35564853556485354 0.3695198329853862
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step
0.0132404181184669 0.3514644351464435 0.3089770354906054
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step
0.21393728222996516 0.26359832635983266 0.267223382045929
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step
0.2397212543554007 0.2510460251046025 0.2359081419624217
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step
0.2397212543554007 0.2594142259414226 0.2359081419624217
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step
15/15 ━━━━━━━

In [15]:
np.log(1)

0.0